In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Import all files to notebook

In [2]:
train = pd.read_csv(r'..\data\house prediction\train.csv',index_col = 'Id')
test = pd.read_csv(r'..\data\house prediction\test.csv',index_col = 'Id')
ans = pd.read_csv(r'..\data\house prediction\sample_submission.csv',index_col = 'Id')

In [3]:
# test = pd.merge(test,ans,on= 'Id')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [5]:
null = []
for i in train.columns:
    val = pd.value_counts(train[i].isna())
    if val[0]/len(train) <= 0.6:
        null.append(i)

print("The columns must delete are : ",null)

The columns must delete are :  ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']


In [101]:
train_delete_col = train.drop(columns=null)
test_delete_col = test.drop(columns=null)

In [105]:
# train_clean = train_delete_col.fillna(np.median)
# test_clean = test_delete_col.fillna(np.median)
train_clean = train_delete_col.fillna(train_delete_col.median())
test_clean = test_delete_col.fillna(train_delete_col.median())

In [108]:
# change object to float
train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [125]:
object_col = train_clean.select_dtypes(include='object')

In [126]:
object_col.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   Street         1460 non-null   object
 2   LotShape       1460 non-null   object
 3   LandContour    1460 non-null   object
 4   Utilities      1460 non-null   object
 5   LotConfig      1460 non-null   object
 6   LandSlope      1460 non-null   object
 7   Neighborhood   1460 non-null   object
 8   Condition1     1460 non-null   object
 9   Condition2     1460 non-null   object
 10  BldgType       1460 non-null   object
 11  HouseStyle     1460 non-null   object
 12  RoofStyle      1460 non-null   object
 13  RoofMatl       1460 non-null   object
 14  Exterior1st    1460 non-null   object
 15  Exterior2nd    1460 non-null   object
 16  MasVnrType     1452 non-null   object
 17  ExterQual      1460 non-null   object
 18  ExterCond      1460 non-null

In [122]:
from sklearn.preprocessing import LabelEncoder
def encoder(data):
    encoder_data = []
    le = LabelEncoder()
    for i in data.columns:
        encoder_data.append(le.fit_transform(data[i]))
    return np.transpose(encoder_data)
#     return pd.DataFrame(np.reshape(np.array(encoder_data),(row,columns)),columns=data_col,index=data_index) 

In [123]:
# train_obj_encoder = encoder(train_clean[object_col])
#test_obj_encoder = encoder(test_clean[object_col])
encoder(train_clean[object_col])
# le = LabelEncoder()
# le.fit_transform(train_clean['Street'])

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['float', 'str']

In [124]:
train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [40]:
train_completely_clean = pd.merge(train_clean.drop(columns = object_col),train_obj_encoder,how='inner',on = 'Id')

NameError: name 'train_obj_encoder' is not defined

In [ ]:
test_obj_encoder = encoder(test_clean[object_col])
test_completely_clean = pd.merge(test_clean.drop(columns = object_col),test_obj_encoder,how='inner',on = 'Id')

In [ ]:
data_corr = train_completely_clean.corr()
key = data_corr['SalePrice'][data_corr.SalePrice>0.3].keys()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
X_train = train_completely_clean[['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF']]
y_train = train_completely_clean['SalePrice']

X_test = test_completely_clean[['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF']]
y_test = test_completely_clean['SalePrice']

In [ ]:
data_corr['SalePrice'][data_corr.SalePrice>0.3]

In [ ]:
lm = LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
y_pred = lm.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error , mean_absolute_error ,r2_score
case1_mse = mean_squared_error(y_test,y_pred)
case1_mae = mean_absolute_error(y_test,y_pred)
case1_r2  = r2_score(y_test,y_pred)
print(case1_mse)
print(case1_mae)
print(case1_r2)

# Linear regression with all variable

In [ ]:
X_train_allvar = train_completely_clean.drop(columns='SalePrice')
y_train_allvar = train_completely_clean['SalePrice']

X_test_allvar = test_completely_clean.drop(columns='SalePrice')
y_test_allvar = test_completely_clean['SalePrice']

In [ ]:
from sklearn.preprocessing import StandardScaler,Normalizer
scale = StandardScaler()
X_train_allvar = scale.fit_transform(X_train_allvar)
X_test_allvar  = scale.fit_transform(X_test_allvar)

In [ ]:
lm_allvar = LinearRegression()
lm_allvar.fit(X_train_allvar,y_train_allvar)

In [ ]:
y_pred_allvar = lm_allvar.predict(X_test_allvar)

In [ ]:
case2_mse = mean_squared_error(y_test_allvar,y_pred_allvar)
case2_mae = mean_absolute_error(y_test_allvar,y_pred_allvar)
case2_r2 = r2_score(y_pred_allvar,y_test_allvar)
print(case2_mse)
print(case2_mae)
print(case2_r2)

### Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
tree.fit(X_train_allvar,y_train_allvar)
ytree_pred = tree.predict(X_test_allvar)
tree_mse = mean_squared_error(ytree_pred,y_test_allvar)
tree_mae = mean_absolute_error(ytree_pred,y_test_allvar)
tree_r2 = r2_score(ytree_pred,y_test_allvar)
print(tree_mse)
print(tree_mae)
print(tree_r2)